In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

from because.probability import prob

hbe:  0.9490824254306831


In [3]:
df = pd.read_csv('/home/arun/Desktop/dataset/Housing/housing.csv', header=0)
df.shape

(384977, 22)

In [4]:
df.columns

Index(['id', 'url', 'region', 'region_url', 'price', 'type', 'sqfeet', 'beds',
       'baths', 'cats_allowed', 'dogs_allowed', 'smoking_allowed',
       'wheelchair_access', 'electric_vehicle_charge', 'comes_furnished',
       'laundry_options', 'parking_options', 'image_url', 'description', 'lat',
       'long', 'state'],
      dtype='object')

In [6]:
#  print first 5 rows of type column
df['type'].unique()

array(['apartment', 'duplex', 'house', 'condo', 'flat', 'townhouse',
       'manufactured', 'loft', 'cottage/cabin', 'in-law', 'land',
       'assisted living'], dtype=object)

In [7]:
#  label encoder for type
le = LabelEncoder()
df['type'] = le.fit_transform(df['type'])


In [8]:
#  print first 5 rows of type column
df['type'].unique()

array([ 0,  4,  6,  2,  5, 11, 10,  9,  3,  7,  8,  1])

In [9]:
df = df[np.logical_and(df.price >= 500, df.price <= 3000)]
df = df[np.logical_and(df.sqfeet >= 500, df.sqfeet <= 3000)]
df = df[np.logical_and(df.beds >= 1, df.beds <= 4)]
df = df[np.logical_and(df.baths >= 1, df.baths <= 4)]

# df = df[df.type == 'house']
df = df[['price', 'sqfeet', 'beds', 'baths', 'type']]

print(df.shape)
df.describe()

(348244, 5)


,price,sqfeet,beds,baths,type
count,348244.000000,348244.000000,348244.000000,348244.000000,348244.000000
mean,1166.078118,998.990647,1.924711,1.491189,1.183900
std,461.854198,320.029079,0.765155,0.545420,2.886026
min,500.000000,500.000000,1.000000,1.000000,0.000000
25%,830.000000,775.000000,1.000000,1.000000,0.000000
50%,1050.000000,950.000000,2.000000,1.000000,0.000000
75%,1395.000000,1146.000000,2.000000,2.000000,0.000000
max,3000.000000,3000.000000,4.000000,4.000000,11.000000


In [10]:
# converting to dictionary format
dfDict = df.to_dict('list')

# print keys of the dictionary
print(dfDict.keys())


dict_keys(['price', 'sqfeet', 'beds', 'baths', 'type'])


In [ ]:
# dfDict

In [18]:
prob.DEBUG = True

In [12]:
# Instantiate prob.ProbSpace with that dataset
myPS = prob.ProbSpace(dfDict)

In [13]:
print(myPS.dependence('price', 'sqfeet'))
print(myPS.dependence('price', 'sqfeet',dMethod= 'rcot'))
print(myPS.dependence('price', 'sqfeet',dMethod= 'prob'))

1.0000000000001072
1.0000000000000153
0.99999999105738


In [24]:

myPS.dependence('price', 'beds', ['type', 'sqfeet'], dMethod='rcot')

1.0000000000000306

In [11]:
myPS.dependence('beds', 'price')

1.0000000000001839

In [12]:
prob.DEBUG = False
print(myPS.E('price'))
print(myPS.E('sqfeet'))

print(myPS.distr('price'))
prob.DEBUG = True

1166.0781176416535
998.9906473621943


In [26]:
myPS.isIndependent('price', 'beds')

False

In [27]:
d = myPS.distr('price')

ProbSpace.distr: P( price | [] )


In [28]:
p = list(dir(d))
len(p)

75

In [29]:
for i in p:
    if not i.startswith('__'):
        print(i)

E
N
P
Prangex
R
SetHistogram
ToHistTuple
ToHistogram
binCount
binValue
bins
compare
compare2
compare3
compare4
compare5
compare_ks_old
data
filters
getBin
getBinForVal
isDiscrete
isNull
ksAlpha
ksThreshold
kurtCache
kurtosis
max
maxVal
mean
meanCache
median
min
minVal
modality
mode
mvData
percentile
rvName
skew
skewCache
stDev
stats
style
var
varCache


In [30]:
print(d.percentile(1))

525.2174260975337


In [31]:
vars = ['price','beds','sqfeet']
lim = 1
dims = 3
numPts = 20

distrs = [myPS.distr(c) for c in vars]    # Get a list of distribution objects for each variable
minvs = [distr.percentile(lim) for distr in distrs]
maxvs = [distr.percentile(100-lim) for distr in distrs]
incrs = [(maxvs[i] - minvs[i]) / (numPts-1) for i in range(dims)]

ProbSpace.distr: P( price | [] )
ProbSpace.distr: P( beds | [] )
ProbSpace.distr: P( sqfeet | [] )


In [32]:
minvs

[525.2174260975337, 1.0, 529.916362025036]

In [34]:
maxvs

[2702.2952196043443, 4.0, 2187.6233521657264]

In [35]:
incrs

[114.58304176351636, 0.15789473684210525, 87.24773632319423]

In [36]:
ps = myPS
vSpaces = []
incrs = []
for i in range(dims):
    var = vars[i]
    if ps.isDiscrete(var):
        vSpace = ps.getMidpoints(var)
        incrs.append(1)
    else:
        vSpace = list(np.linspace(minvs[i], maxvs[i], numPts))
        incrs.append(vSpace[1] - vSpace[0])
    vSpaces.append(vSpace)

In [37]:
incrs


[114.58304176351635, 1, 87.2477363231942]

In [38]:
vSpaces

[[525.2174260975337,
  639.80046786105,
  754.3835096245664,
  868.9665513880827,
  983.5495931515991,
  1098.1326349151154,
  1212.715676678632,
  1327.2987184421481,
  1441.8817602056647,
  1556.464801969181,
  1671.0478437326974,
  1785.6308854962138,
  1900.21392725973,
  2014.7969690232464,
  2129.380010786763,
  2243.963052550279,
  2358.5460943137955,
  2473.129136077312,
  2587.712177840828,
  2702.2952196043443],
 [1.0, 2.0, 3.0, 4.0],
 [529.916362025036,
  617.1640983482303,
  704.4118346714245,
  791.6595709946188,
  878.907307317813,
  966.1550436410072,
  1053.4027799642015,
  1140.6505162873957,
  1227.89825261059,
  1315.145988933784,
  1402.3937252569783,
  1489.6414615801727,
  1576.889197903367,
  1664.1369342265612,
  1751.3846705497554,
  1838.6324068729496,
  1925.8801431961438,
  2013.127879519338,
  2100.375615842532,
  2187.6233521657264]]

In [ ]:
def findPoint(minv, maxv, indx):
    val = ((maxv - minv) / (numPts) ) * indx + minv
    return val

gridSet = []

for i in range(20):
    y = (i-1) % 20
    gridSet.append(findPoint(minvs[0], maxvs[0], y))